### **INITIALIZATION:**
- I use these three lines of code on top of my each notebooks because it will help to prevent any problems while reloading the same project. And the third line of code helps to make visualization within the notebook.


In [1]:
#@ INITIALIZATION:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

**DOWNLOADING LIBRARIES AND DEPENDENCIES:**
- I have downloaded all the libraries and dependencies required for the project in one particular cell.

In [4]:
#@ DOWNLOADING THE LIBRARIES AND DEPENDENCIES:
# !pip install -U d2l
from d2l import torch as d2l

import os
import torch     
from torch import nn                                
from IPython import display

**GETTING THE DATASET:**
- I have used google colab for this project so the process of downloading and reading the data might be different in other platforms. I will use [**Large Movie Review Dataset**](https://ai.stanford.edu/~amaas/data/sentiment/) for this project. The dataset is divided into training and testing and each contains 25000 movie reviews. 

In [6]:
#@ GETTING THE DATASET: 
batch_size = 64                                                     # Initializing Batch Size. 
train_iter, test_iter, vocab = d2l.load_data_imdb(batch_size)       # Initializing Training and Test Iterations. 

### **RECURRENT NEURAL NETWORK MODEL:**
- Each words obtains a feature vector from the embedding layer which is further encoded using bidirectional RNN to obtain sequence information. Here the `Embedding` instance is the embedding layer, the `LSTM` instance is the hidden layer for sequence encoding and the `Dense` instance is the output layer for generated classification result. 

In [7]:
#@ INITIALIZING RECURRENT NEURAL NETWORK MODEL: 
class BiRNN(nn.Module):                                                       # Initializing Bidirectional RNN. 
  def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, 
               **kwargs):                                                     # Initializing Constructor Function. 
    super(BiRNN, self).__init__(**kwargs)
    self.embedding = nn.Embedding(vocab_size, embed_size)                     # Initializing Embedding Layer. 
    self.encoder = nn.LSTM(embed_size, num_hiddens, num_layers=num_layers, 
                           bidirectional=True)                                # Initializing Bidirectional LSTM. 
    self.decoder = nn.Linear(4*num_hiddens, 2)                                # Initializing Linear Layer. 
  
  def forward(self, inputs):                                                  # Forward Propagation Function. 
    embeddings = self.embedding(inputs.T)                                     # Implementation of Embedding Layer. 
    self.encoder.flatten_parameters()    
    outputs, _ = self.encoder(embeddings)                                     # Implementation of LSTM. 
    encoding = torch.cat((outputs[0], outputs[-1]), dim=1)                    # Concatenating Initial and Final Timestep. 
    outs = self.decoder(encoding)                                             # Implementation of Linear Layer. 
    return outs

In [8]:
#@ IMPLEMENTATION OF RECURRENT NEURAL NETWORKS: 
embed_size, num_hiddens = 100, 100                              # Initialization of Parameters. 
num_layers, devices = 2, d2l.try_all_gpus()                     # Initialization of Parameters. 
net = BiRNN(len(vocab), embed_size, num_hiddens, num_layers)    # Initialization of Bidirectional RNN Model. 

#@ INITIALIZATION OF WEIGHTS: 
def init_weights(m):                                            # Function for Initializing Weights. 
  if type(m) == nn.Linear:
    nn.init.xavier_uniform_(m.weight)                           # Xavier Initialization. 
  if type(m) == nn.LSTM:
    for param in m._flat_weights_names:
      if "weight" in param:
        nn.init.xavier_uniform_(m._parameters[param])           # Xavier Initialization. 
net.apply(init_weights);                                        # Initializing Weights. 